<a href="https://colab.research.google.com/github/PyEatingContest/corrosive-stones/blob/master/STEVE_piper_multilingual_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["TORCH_FORCE_WEIGHTS_ONLY_LOAD"] = "0"

In [ ]:
# 1. Downgrade Python to 3.10
!sudo apt-get update
!sudo apt-get install python3.10 python3.10-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10

# 2. Fix the "No module named pip" error by manually installing pip for 3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3

# 3. Verify versions - you should see Python 3.10.x and a working pip
!python3 --version
!pip3 --version

# 4. Fix Matplotlib for headless environment
import matplotlib
matplotlib.use('Agg', force=True)
import os
os.environ["MPLBACKEND"] = "Agg"

# 5. Install Piper dependencies using the newly fixed pip
!pip3 install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy==1.26 onnxruntime>=1.15.0 pytorch-lightning==1.9.0

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,316 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:13 http://archive.ubuntu.com/ubu

In [ ]:
import torch
import pathlib

# Allow PyTorch to load the specific 'pathlib' objects used in Piper checkpoints
torch.serialization.add_safe_globals([pathlib.PosixPath, pathlib.PurePosixPath])

In [ ]:
#@markdown ## <font color="ffc800"> **Google Colab Anti-Disconnect.** 🔌
#@markdown ---
#@markdown #### Avoid automatic disconnection. Still, it will disconnect after <font color="orange">**6 to 12 hours**</font>.

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [ ]:
#@markdown # <font color="ffc800"> **Mount Google Drive.** 📂
#@markdown ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 1. Clone the repository so the source files are present on the local disk
!git clone https://github.com/rhasspy/piper.git /content/piper

# 2. Navigate to the python source directory
%cd /content/piper/src/python

Cloning into '/content/piper'...
remote: Enumerating objects: 2039, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 2039 (delta 131), reused 125 (delta 125), pack-reused 1887 (from 3)
Receiving objects: 100% (2039/2039), 213.36 MiB | 36.92 MiB/s, done.
Resolving deltas: 100% (1123/1123), done.
/content/piper/src/python


In [ ]:
import torch
import pathlib
from functools import wraps

# Patch the Piper main file directly to allow 'untrusted' checkpoint loading
# We inject this at the top of the actual script so it persists through the run.
file_path = '/content/piper/src/python/piper_train/__main__.py'
patch_code = """
import torch
import pathlib
from functools import wraps
torch.serialization.add_safe_globals([pathlib.PosixPath, pathlib.PurePosixPath])
_orig_load = torch.load
@wraps(_orig_load)
def _patched_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return _orig_load(*args, **kwargs)
torch.load = _patched_load
"""

with open(file_path, 'r') as f:
    content = f.read()
with open(file_path, 'w') as f:
    f.write(patch_code + "\n" + content)

print("✅ PyTorch 2.6 security patch successfully applied to Piper source.")

✅ PyTorch 2.6 security patch successfully applied to Piper source.


In [ ]:
# 1. Navigate to the core source directory
%cd /content/piper/src/python

# 2. Build the monotonic_align extension
# This requires build-essential and python3-dev (usually pre-installed on Colab)
!bash build_monotonic_align.sh

/content/piper/src/python


In [ ]:
# Uninstall the problematic new versions
!pip uninstall -y torch torchvision torchaudio pytorch-lightning lightning onnxscript onnxruntime onnx

# Install the 'Golden Trio' for Piper exports
!pip install torch==2.1.0 pytorch-lightning==1.9.0 torchmetrics==0.11.4 onnx==1.16.1 onnxruntime==1.17.1 -q

# Patch 1: The math assertion
!sed -i 's/assert (discriminant >= 0).all(), discriminant/# assert (discriminant >= 0).all(), discriminant/' /content/piper/src/python/piper_train/vits/transforms.py

# Patch 2: The mask guard (adding .detach() helps the tracer ignore the shape check)
!sed -i 's/h = self.pre(x0) \* x_mask/h = self.pre(x0) * x_mask.detach()/' /content/piper/src/python/piper_train/vits/modules.py

Found existing installation: torch 2.10.0
Uninstalling torch-2.10.0:
  Successfully uninstalled torch-2.10.0
Found existing installation: pytorch-lightning 1.9.0
Uninstalling pytorch-lightning-1.9.0:
  Successfully uninstalled pytorch-lightning-1.9.0
Found existing installation: onnxruntime 1.23.2
Uninstalling onnxruntime-1.23.2:
  Successfully uninstalled onnxruntime-1.23.2


In [ ]:
import os

ckpt_to_export = "/content/drive/MyDrive/colab/piper/lightning_logs/version_4/checkpoints/epoch=2744-step=2024.ckpt"
output_onnx = "/content/drive/MyDrive/colab/piper/steven_voice_2744.onnx"

# We use the shell command but set an environment variable to force legacy behavior
!export TORCH_ONNX_LOW_LEVEL_DEBUG=1 && \
 python3 -m piper_train.export_onnx "{ckpt_to_export}" "{output_onnx}"

if os.path.exists(output_onnx):
    print(f"✅ SUCCESS! File created: {output_onnx}")
    print(f"📦 Size: {os.path.getsize(output_onnx) / 1024**2:.2f} MB")
    !cp /content/drive/MyDrive/colab/piper/config.json "{output_onnx}.json"
else:
    print("❌ Export still failed. If you see a 'Traceback', paste the very last line here.")

/usr/local/lib/python3.10/dist-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Removing weight norm...
/content/piper/src/python/piper_train/vits/attentions.py:235: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not ge

In [ ]:
import os

export_script_path = '/content/piper/src/python/piper_train/export_onnx.py'

# This version checks if it's already patched to avoid infinite recursion
export_patch = """
import torch
import pathlib
from functools import wraps

# Anti-recursion safety: Only patch if torch.load isn't already our patched version
if not hasattr(torch, '_is_piper_patched'):
    torch.serialization.add_safe_globals([pathlib.PosixPath, pathlib.PurePosixPath])

    _orig_load = torch.load
    @wraps(_orig_load)
    def _patched_load(*args, **kwargs):
        kwargs['weights_only'] = False
        return _orig_load(*args, **kwargs)

    torch.load = _patched_load
    torch._is_piper_patched = True
"""

# Read the current content
with open(export_script_path, 'r') as f:
    current_content = f.read()

# Only apply the patch if it's not already in the file
if "_is_piper_patched" not in current_content:
    with open(export_script_path, 'w') as f:
        f.write(export_patch + current_content)
    print("✅ Export script patched (with recursion protection).")
else:
    print("ℹ️ Export script was already patched. Skipping to prevent recursion error.")

In [ ]:
import os

export_script_path = '/content/piper/src/python/piper_train/export_onnx.py'

# This version checks if it's already patched to avoid infinite recursion
export_patch = """
import torch
import pathlib
from functools import wraps

# Anti-recursion safety: Only patch if torch.load isn't already our patched version
if not hasattr(torch, '_is_piper_patched'):
    torch.serialization.add_safe_globals([pathlib.PosixPath, pathlib.PurePosixPath])

    _orig_load = torch.load
    @wraps(_orig_load)
    def _patched_load(*args, **kwargs):
        kwargs['weights_only'] = False
        return _orig_load(*args, **kwargs)

    torch.load = _patched_load
    torch._is_piper_patched = True
"""

# Read the current content
with open(export_script_path, 'r') as f:
    current_content = f.read()

# Only apply the patch if it's not already in the file
if "_is_piper_patched" not in current_content:
    with open(export_script_path, 'w') as f:
        f.write(export_patch + current_content)
    print("✅ Export script patched (with recursion protection).")
else:
    print("ℹ️ Export script was already patched. Skipping to prevent recursion error.")

In [ ]:
!pip install onnxscript onnxruntime onnx

In [ ]:
# This comments out the line that is causing the TorchExportError
!sed -i 's/assert (discriminant >= 0).all(), discriminant/# assert (discriminant >= 0).all(), discriminant/' /content/piper/src/python/piper_train/vits/transforms.py

print("✅ Patched transforms.py to bypass the export guard.")

✅ Patched transforms.py to bypass the export guard.


In [ ]:
# This patches the specific line in modules.py identified in your error log
!sed -i '449s/h = self.pre(x0) \* x_mask/# h = self.pre(x0) * x_mask/' /content/piper/src/python/piper_train/vits/modules.py
# Note: Since the error suggested inserting a check, we can also try to simply
# satisfy the exporter by forcing a dummy check, but commenting out is often cleaner for export.

In [ ]:
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
!pip install onnxscript==0.1.0 onnxruntime==1.16.3 onnx==1.15.0––

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 12.6 MB/s  0:00:53
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 156.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 54.9 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.4 MB/s  0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.6.0
    Uninstalling triton-3.6.0:
      Successfully uninstalled triton-3.6.0
  Attempting uninstall: torch
    Found existing installation: torch 2.10.0
    Uninstalling torch-2.10.0:
      Successfully uninstalled torch-2.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [torchaudio]
INFO: pip is looking at multiple versions of onnxscript to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install onnx==1.15.0 and onnxscrip

In [ ]:
# 1. Clean up and install only what we need for legacy export
!pip uninstall -y onnxscript onnxruntime onnx
!pip install onnx==1.16.1 onnxruntime==1.17.1

# 2. Patch the Piper code (One last time to be sure)
# This removes the assertion and the mask guard that caused your previous u14/u18 errors
!sed -i 's/assert (discriminant >= 0).all(), discriminant/# assert (discriminant >= 0).all(), discriminant/' /content/piper/src/python/piper_train/vits/transforms.py
!sed -i 's/h = self.pre(x0) \* x_mask/h = self.pre(x0) * x_mask.detach()/' /content/piper/src/python/piper_train/vits/modules.py

# 3. Force the Legacy Export Path
# We set dynamo=False to tell PyTorch "Stop trying to be smart, just trace the graph."
import torch
import sys
sys.path.append("/content/piper/src/python")
from piper_train.export_onnx import main

# Re-run the export but we'll modify the command to ensure dynamo is OFF
ckpt_to_export = "/content/drive/MyDrive/colab/piper/lightning_logs/version_4/checkpoints/epoch=2744-step=2024.ckpt"
output_onnx = "/content/drive/MyDrive/colab/piper/steven_voice_2744.onnx"

# Run the export script directly via shell to keep it simple
!python3 -m piper_train.export_onnx "{ckpt_to_export}" "{output_onnx}"

Found existing installation: onnxscript 0.5.7
Uninstalling onnxscript-0.5.7:
  Successfully uninstalled onnxscript-0.5.7
Found existing installation: onnxruntime 1.23.2
Uninstalling onnxruntime-1.23.2:
  Successfully uninstalled onnxruntime-1.23.2
Found existing installation: onnx 1.20.1
Uninstalling onnx-1.20.1:
  Successfully uninstalled onnx-1.20.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 45.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [onnxruntime]


ModuleNotFoundError: No module named 'pytorch_lightning'

In [ ]:
*** OLDER CODE BELOW ***

In [ ]:
# Clean up any redisual older piper code
!rm -rf /content/piper

# Load an older python to run the training
!pip install -q condacolab
!pip install matplotlib
import condacolab
condacolab.install()

import os
import matplotlib

# 1. Clear any existing backend environment variable
os.environ.pop("MPLBACKEND", None)

# 2. Reset Matplotlib to its defaults
matplotlib.rcdefaults()

# 3. Force the 'Agg' backend (headless/non-GUI)
matplotlib.use('Agg', force=True)

# 4. Verify the change
print(f"Current Matplotlib backend: {matplotlib.get_backend()}")

✨🍰✨ Everything looks OK!
Current Matplotlib backend: Agg


# <font color="ffc800">🔧 ***First steps.*** 🔧

In [ ]:
#@markdown ## <font color="ffc800"> **Check GPU type.** 👁️
#@markdown ---
#@markdown #### A higher capable GPU can lead to faster training speeds. By default, you will have a <font color="orange">**Tesla T4**</font>.
!nvidia-smi

Sun Jan 25 22:43:49 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#@markdown # <font color="ffc800"> **Install software.** 📦
#@markdown ---
#@markdown ####In this cell the synthesizer and its necessary dependencies to execute the training will be installed. (this may take a while)

# clone:
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
#!pip install -q -r requirements.txt
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy==1.26 onnxruntime>=1.15.0 pytorch-lightning
!pip install --upgrade gdown transformers
!bash build_monotonic_align.sh
# Useful vars:
use_whisper = True
print("Done!")

/content/piper/src/python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.7 MB/s eta 0:00:00
Compiling /content/piper/src/python/piper_train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /content/piper/src/python/piper_train/vits/monotonic_align/core.pyx
performance hint: core.pyx:5:0: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:3

In [ ]:
#@markdown # <font color="ffc800"> **Install older cross-compatible version of software.** 📦
# 1. Install the cross-compatible version
!pip install piper-phonemize-cross

# 2. Re-install Piper without the strict dependency check
%cd /content/piper/src/python
!pip install -e . --no-deps
!pip install pytorch-lightning==1.9.0 onnxruntime

/content/piper/src/python
Obtaining file:///content/piper/src/python
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
# Check thank the machine can access the piper modules
import sys
sys.path.append('/content/piper/src/python')

try:
    from piper_train.vits import commons
    print("✅ Piper training modules are now accessible!")
except ImportError as e:
    print(f"❌ Still missing something: {e}")

❌ Still missing something: No module named 'piper_train'


In [ ]:
import os

file_path = '/content/piper/src/python/piper_train/__main__.py'

# The code we want to inject at the very top of the file
patch_code = """
import torch
import pathlib
from functools import wraps

# Add specific safe globals
torch.serialization.add_safe_globals([pathlib.PosixPath, pathlib.PurePosixPath])

# Force weights_only=False globally for this process
_orig_load = torch.load
@wraps(_orig_load)
def _patched_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return _orig_load(*args, **kwargs)
torch.load = _patched_load
"""

with open(file_path, 'r') as f:
    original_content = f.read()

# Write the patch at the top
with open(file_path, 'w') as f:
    f.write(patch_code + "\n" + original_content)

print("✅ Global Torch patch applied to Piper source.")

✅ Global Torch patch applied to Piper source.


In [ ]:
#@markdown # <font color="ffc800"> **Export a trained version of the model.** 📦

# Define paths for your specific checkpoint and output
ckpt_path = "/content/drive/MyDrive/colab/piper/lightning_logs/version_4/checkpoints/epoch=2744-step=2024.ckpt"
output_onnx = "/content/drive/MyDrive/colab/piper/steven_voice_2744.onnx"

# Run the export module
!python3 -m piper_train.export_onnx \
    "{ckpt_path}" \
    "{output_onnx}"

# Ensure the config file is also copied and named correctly
!cp /content/drive/MyDrive/colab/piper/config.json "{output_onnx}.json"

print(f"✅ Export complete! Your files are ready in Drive: \n1. {output_onnx} \n2. {output_onnx}.json")

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/piper/src/python/piper_train/export_onnx.py", line 111, in <module>
    main()
  File "/content/piper/src/python/piper_train/export_onnx.py", line 42, in main
    model = VitsModel.load_from_checkpoint(args.checkpoint, dataset=None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pytorch_lightning/core/saving.py", line 139, in load_from_checkpoint
    return _load_from_checkpoint(
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pytorch_lightning/core/saving.py", line 160, in _load_from_checkpoint
    checkpoint = pl_load(checkpoint_path, map_location=map_location)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/lightning_fabric/utilities/cloud_io.py", line 4

# <font color="ffc800"> 🤖 ***Training.*** 🤖

In [ ]:
#@markdown # <font color="ffc800"> **1. Extract dataset.** 📥
#@markdown ---
#@markdown ####Important: the audios must be in <font color="orange">**wav format, (16000 or 22050hz, 16-bits, mono), and, for convenience, numbered. Example:**

#@markdown * <font color="orange">**1.wav**</font>
#@markdown * <font color="orange">**2.wav**</font>
#@markdown * <font color="orange">**3.wav**</font>
#@markdown * <font color="orange">**.....**</font>

#@markdown ---
import os
import wave
import zipfile
import datetime

def get_dataset_duration(wav_path):
    totalduration = 0
    for file_name in [x for x in os.listdir(wav_path) if os.path.isfile(x) and ".wav" in x]:
        with wave.open(file_name, "rb") as wave_file:
            frames = wave_file.getnframes()
            rate = wave_file.getframerate()
            duration = frames / float(rate)
            totalduration += duration
    wav_count = len(os.listdir(wav_path))
    duration_str = str(datetime.timedelta(seconds=round(totalduration, 0)))
    return wav_count, duration_str

%cd /content
if not os.path.exists("/content/dataset"):
    os.makedirs("/content/dataset")
    os.makedirs("/content/dataset/wavs")
%cd /content/dataset
#@markdown ### Audio dataset path to unzip:
zip_path = "/content/drive/MyDrive/piper_dataset.zip" #@param {type:"string"}
zip_path = zip_path.strip()
if zip_path:
    if os.path.exists(zip_path):
        if zipfile.is_zipfile(zip_path):
            print("Unzipping audio content...")
            !unzip -q -j "{zip_path}" -d /content/dataset/wavs
        else:
            print("Copying audio contents of this folder...")
            fp = zip_path + "/."
            !cp -a "$fp" "/content/dataset/wavs"
    else:
        raise Exception("The path provided to the wavs is not correct. Please set a valid path.")
else:
    raise Exception("You must provide with a path to the wavs.")
if os.path.exists("/content/dataset/wavs/wavs"):
    for file in os.listdir("/content/dataset/wavs/wavs"):
        !mv /content/dataset/wavs/wavs/"$file"  /content/dataset/wavs/"$file"
    !rm -r /content/dataset/wavs/*.txt
    !rm -r /content/dataset/wavs/*.csv
%cd /content/dataset/wavs
audio_count, dataset_dur = get_dataset_duration("/content/dataset/wavs")
print(f"Opened dataset with {audio_count} wavs with duration {dataset_dur}.")
%cd ..
#@markdown ---

/content
/content/dataset
Unzipping audio content...
replace /content/dataset/wavs/0000000135.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
/content/dataset/wavs
Opened dataset with 151 wavs with duration 0:20:23.
/content/dataset


In [ ]:
#@markdown # <font color="ffc800"> **2. Upload the transcript file.** 📝
#@markdown ---
#@markdown ####<font color="orange">**Important: the transcription means writing what the character says in each of the audios, and it must have the following structure:**

#@markdown ##### <font color="orange">For a single-speaker dataset:
#@markdown * wavs/1.wav|This is what my character says in audio 1.
#@markdown * wavs/2.wav|This, the text that the character says in audio 2.
#@markdown * ...

#@markdown ##### <font color="orange">For a multi-speaker dataset:

#@markdown * wavs/speaker1audio1.wav|speaker1|This is what the first speaker says.
#@markdown * wavs/speaker1audio2.wav|speaker1|This is another audio of the first speaker.
#@markdown * wavs/speaker2audio1.wav|speaker2|This is what the second speaker says in the first audio.
#@markdown * wavs/speaker2audio2.wav|speaker2|This is another audio of the second speaker.
#@markdown * ...

#@markdown And so on. In addition, the transcript must be in a <font color="orange">**.csv or .txt format. (UTF-8 without BOM)**

#@markdown ## Auto-transcribe with whisper if transcription is not provided.

#@markdown **Note: If you don't upload any transcription files, the wavs will be transcribed using the whisper tool when you execute the next step. Then, the notebook will continue with the rest of the preprocessing if there are no errors. Although the Whisper tool has good transcription results, in my experience I recommend transcribing manually and uploading it from this cell, since a good TTS voice needs to be optimized to give even better results. For example, when transcribing manually you will be able to observe every detail that the speaker makes (such as punctuation, sounds, etc.), and capture them in the transcription according to the speaker's intonations.**


#@markdown However, if you want to transcribe and review this transcription, you can use the individual notebooks:

#@markdown * [English](http://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/OpenAI_Whisper_-_DotCSV_(Speech_dataset_multi-transcryption_support)en.ipynb)
#@markdown * [French](http://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/OpenAI_Whisper_-_DotCSV_(Speech_dataset_multi-transcryption_support)fr.ipynb)
#@markdown * [Spanish](http://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/OpenAI_Whisper_-_DotCSV_(Speech_dataset_multi-transcryption_support)es.ipynb)

#@markdown ---
%cd /content/dataset
from google.colab import files
!rm /content/dataset/metadata.csv

if os.path.exists("/content/dataset/wavs/_transcription.txt"):
  !mv "/content/dataset/wavs/_transcription.txt" metadata.csv
else:
  listfn, length = files.upload().popitem()
  if listfn != "metadata.csv":
    !mv "$listfn" metadata.csv

use_whisper = False
%cd ..

/content/dataset


Saving metadata.csv to metadata.csv
/content


In [ ]:
#@markdown # <font color="ffc800"> **3. Preprocess dataset.** 🔄
#@markdown ---
import os
if use_whisper:
    import torch
    from faster_whisper import WhisperModel
    from tqdm import tqdm
    from google import colab

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    def make_dataset(path, language):
        metadata = ""
        text = ""
        files = [f for f in os.listdir(path) if f.endswith(".wav")]
        assert len(files) > 0, "You don't have wavs uploaded either! Please upload at least one zip with the wavs in step 2."
        metadata_file = open(f"{path}/../metadata.csv", "w")
        whisper = WhisperModel("large-v3", device=device, compute_type="float16")
        for audio_file in tqdm(files):
            full_path = os.path.join(path, audio_file)
            segments, _ = whisper.transcribe(full_path, word_timestamps=False, language=language)
            for segment in segments:
                text += segment.text
            text = text.strip()
            text = text.replace('\n', ' ')
            metadata = f"{audio_file}|{text}\n"
            metadata_file.write(metadata)
            text = ""
        colab.files.download(f"{path}/../metadata.csv")
        del whisper
        return True

#@markdown ### First of all, select the language of your dataset.
language = "English (U.S.)" #@param ["ألعَرَبِي", "Català", "čeština", "Dansk", "Deutsch", "Ελληνικά", "English (British)", "English (U.S.)", "Español (Castellano)", "Español (Latinoamericano)", "Suomi", "Français", "Magyar", "Icelandic", "Italiano", "ქართული", "қазақша", "Lëtzebuergesch", "नेपाली", "Nederlands", "Norsk", "Polski", "Português (Brasil)", "Português (Portugal)", "Română", "Русский", "Српски", "Svenska", "Kiswahili", "Türkçe", "украї́нська", "Tiếng Việt", "简体中文"]
#@markdown ---
# language definition:
languages = {
    "ألعَرَبِي": "ar",
    "Català": "ca",
    "čeština": "cs",
    "Dansk": "da",
    "Deutsch": "de",
    "Ελληνικά": "el",
    "English (British)": "en",
    "English (U.S.)": "en-us",
    "Español (Castellano)": "es",
    "Español (Latinoamericano)": "es-419",
    "Suomi.": "fi",
    "Français": "fr",
    "Magyar": "hu",
    "Icelandic": "is",
    "Italiano": "it",
    "ქართული": "ka",
    "қазақша": "kk",
    "Lëtzebuergesch": "lb",
    "नेपाली": "ne",
    "Nederlands": "nl",
    "Norsk": "nb",
    "Polski": "pl",
    "Português (Brasil)": "pt-br",
    "Português (Portugal)": "pt-pt",
    "Română": "ro",
    "Русский": "ru",
    "Српски": "sr",
    "Svenska": "sv",
    "Kiswahili": "sw",
    "Türkçe": "tr",
    "украї́нська": "uk",
    "Tiếng Việt": "vi",
    "简体中文": "zh"
}

def _get_language(code):
    return languages[code]

final_language = _get_language(language)
#@markdown ### Choose a name for your model:
model_name = "" #@param {type:"string"}
#@markdown ---
# output:
#@markdown ### Choose the working folder: (recommended to save to Drive)

#@markdown The working folder will be used in preprocessing, but also in training the model.
output_path = "/content/drive/MyDrive/colab/piper" #@param {type:"string"}
output_dir = output_path+"/"+model_name
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
#@markdown ---
#@markdown ### Choose dataset format:
dataset_format = "ljspeech" #@param ["ljspeech", "mycroft"]
#@markdown ---
#@markdown ### Is this a single speaker dataset? Otherwise, uncheck:
single_speaker = True #@param {type:"boolean"}
if single_speaker:
  force_sp = " --single-speaker"
else:
  force_sp = ""
#@markdown ---
#@markdown ### Select the sample rate of the dataset:
sample_rate = "22050" #@param ["16000", "22050"]
#@markdown ---
# creating paths:
if not os.path.exists("/content/audio_cache"):
    os.makedirs("/content/audio_cache")
%cd /content/piper/src/python
#@markdown ### Do you want to train using this sample rate, but your audios don't have it?
#@markdown The resampler helps you do it quickly!
resample = False #@param {type:"boolean"}
if resample:
  !python resample.py --input_dir "/content/dataset/wavs" --output_dir "/content/dataset/wavs_resampled" --output_sr {sample_rate} --file_ext "wav"
  !mv /content/dataset/wavs_resampled/* /content/dataset/wavs
#@markdown ---
# check transcription:
if use_whisper:
    print("Transcript file hasn't been uploaded. Transcribing these audios using Whisper...")
    make_dataset("/content/dataset/wavs", final_language[:2])
    print("Transcription done! Pre-processing...")
!python -m piper_train.preprocess \
  --language {final_language} \
  --input-dir /content/dataset \
  --cache-dir "/content/audio_cache" \
  --output-dir "{output_dir}" \
  --dataset-name "{model_name}" \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}
print("Preprocessing done!")

/content/piper/src/python
INFO:preprocess:Single speaker dataset
INFO:preprocess:Wrote dataset config
INFO:preprocess:Processing 150 utterance(s) with 2 worker(s)
Preprocessing done!


In [ ]:
#@markdown # <font color="ffc800"> **4. Settings.** 🧰
#@markdown ---
import json
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
import os
import re
import glob
#@markdown ### <font color="orange">**Select the action to train this dataset: (READ CAREFULLY)**

#@markdown * The option to <font color="orange">continue a training</font> is self-explanatory. If you've previously trained a model with free colab, your time is up and you're considering training it some more, this is ideal for you. You just have to set the same settings that you set when you first trained this model.
#@markdown * The option to <font color="orange">convert a single-speaker model to a multi-speaker model</font> is self-explanatory, and for this it is important that you have processed a dataset that contains text and audio from all possible speakers that you want to train in your model.
#@markdown * The <font color="orange">finetune</font> option is used to train a dataset using a pretrained model, that is, train on that data. This option is ideal if you want to train a very small dataset (more than five minutes recommended).
#@markdown * The <font color="orange">train from scratch</font> option builds features such as dictionary and speech form from scratch, and this may take longer to converge. For this, hours of audio (8 at least) are recommended, which have a large collection of phonemes.

action = "finetune" #@param ["Continue training", "convert single-speaker to multi-speaker model", "finetune", "train from scratch"]
#@markdown ---
if action == "Continue training":
    checkpoints = glob.glob(f"{output_dir}/lightning_logs/**/checkpoints/last.ckpt", recursive=True)
    if len(checkpoints):
        last_checkpoint = sorted(checkpoints, key=lambda x: int(re.findall(r'version_(\d+)', x)[0]))[-1]
        ft_command = f'--resume_from_checkpoint "{last_checkpoint}" '
        print(f"Continuing {model_name}'s training at: {last_checkpoint}")
    else:
        raise Exception("Training cannot be continued as there is no checkpoint to continue at.")
elif action == "finetune":
    if os.path.exists(f"{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt"):
        raise Exception("Oh no! You have already trained this model before, you cannot choose this option since your progress will be lost, and then your previous time will not count. Please select the option to continue a training.")
    else:
        ft_command = '--resume_from_checkpoint "/content/pretrained.ckpt" '
elif action == "convert single-speaker to multi-speaker model":
    if not single_speaker:
        ft_command = '--resume_from_single_speaker_checkpoint "/content/pretrained.ckpt" '
    else:
        raise Exception("This dataset is not a multi-speaker dataset!")
else:
    ft_command = ""
if action== "convert single-speaker to multi-speaker model" or action == "finetune":
    try:
        with open('/content/piper/notebooks/pretrained_models.json') as f:
            pretrained_models = json.load(f)
        if final_language in pretrained_models:
            models = pretrained_models[final_language]
            model_options = [(model_name, model_name) for model_name, model_url in models.items()]
            model_dropdown = widgets.Dropdown(description = "Choose pretrained model", options=model_options)
            download_button = widgets.Button(description="Download")
            def download_model(btn):
                model_name = model_dropdown.value
                model_url = pretrained_models[final_language][model_name]
                print("\033[93mDownloading pretrained model...")
                if model_url.startswith("1"):
                    !gdown -q "{model_url}" -O "/content/pretrained.ckpt"
                elif model_url.startswith("https://drive.google.com/file/d/"):
                    !gdown -q "{model_url}" -O "/content/pretrained.ckpt" --fuzzy
                else:
                    !wget -q "{model_url}" -O "/content/pretrained.ckpt"
                model_dropdown.close()
                download_button.close()
                output.clear()
                if os.path.exists("/content/pretrained.ckpt"):
                    print("\033[93mModel downloaded!")
                else:
                    raise Exception("Couldn't download the pretrained model!")
            download_button.on_click(download_model)
            display(model_dropdown, download_button)
        else:
            raise Exception(f"There are no pretrained models available for the language {final_language}")
    except FileNotFoundError:
        raise Exception("The pretrained_models.json file was not found.")
else:
    print("\033[93mWarning: this model will be trained from scratch. You need at least 8 hours of data for everything to work decent. Good luck!")
#@markdown ### Choose batch size based on this dataset:
batch_size = 12 #@param {type:"integer"}
#@markdown ---

#@markdown ### Choose the quality for this model:

#@markdown * x-low - 16Khz audio, 5-7M params
#@markdown * medium - 22.05Khz audio, 15-20 params
#@markdown * high - 22.05Khz audio, 28-32M params
quality = "medium" #@param ["high", "x-low", "medium"]
#@markdown ---
#@markdown ### For how many epochs to save training checkpoints?
#@markdown The larger your dataset, you should set this saving interval to a smaller value, as epochs can progress longer time.
checkpoint_epochs = 5 #@param {type:"integer"}
#@markdown ---
#@markdown ### Interval to save best k models:
#@markdown Set to 0 if you want to disable saving multiple models. If this is the case, check the checkbox below. If set to 1, models will be saved with the file name epoch=xx-step=xx.ckpt, so you will need to empty Drive's trash every so often.
num_ckpt = 1 #@param {type:"integer"}
#@markdown ---
#@markdown ### Save latest model:
#@markdown This checkbox must be checked if you want to save a single model (last.ckpt). Saving a single model is applied only if num_ckpt is equal to 0. If so, the interval parameter of epochs to save is ignored, since the last model per epoch is saved; also, you won't have to worry about storage. Being equal to 1, last.ckpt will be saved, but another model (model_vVersion.ckpt, the latter takes into account the epoch range you set), so you would have to empty the trash often.

#@markdown **It's not recommended to use this option in extremely small datasets, since by saving the last model each epoch, this process will be very fast and the trainer will not be able to save the complete model, which would result in a corrupt last.ckpt.**
save_last = False # @param {type:"boolean"}
#@markdown ---
#@markdown ### Step interval to generate model samples:
log_every_n_steps = 1000 #@param {type:"integer"}
#@markdown ---
#@markdown ### Training epochs:
max_epochs = 3000 #@param {type:"integer"}
#@markdown ---

Model downloaded!


In [ ]:
#@markdown # <font color="ffc800"> **5. Run the TensorBoard extension.** 📈
#@markdown ---
#@markdown The TensorBoard is used to visualize the results of the model while it's being trained such as audio and losses.

%load_ext tensorboard
%tensorboard --logdir {output_dir}

In [ ]:
!pip install matplotlib

In [ ]:
file_path = '/content/piper/src/python/piper_train/vits/lightning.py'

with open(file_path, 'r') as f:
    lines = f.readlines()

new_lines = []
skip_mode = False

for line in lines:
    # Identify the start of the problematic function
    if "def on_validation_end(self) -> None:" in line:
        new_lines.append(line)
        # Inject the safe version
        new_lines.append("        if self.logger is not None and hasattr(self.logger, 'experiment'):\n")
        new_lines.append("            if hasattr(self.logger.experiment, 'add_audio'):\n")
        new_lines.append("                for i, audio in enumerate(self.validation_outputs):\n")
        new_lines.append("                    self.logger.experiment.add_audio(\n")
        new_lines.append("                        f'sample_{i}',\n")
        new_lines.append("                        audio,\n")
        new_lines.append("                        self.global_step,\n")
        new_lines.append("                        sample_rate=self.hparams.audio.sample_rate\n")
        new_lines.append("                    )\n")
        new_lines.append("        return super().on_validation_end()\n")
        skip_mode = True # Skip the original lines until we hit the next function

    # Stop skipping once we hit the next method or end of file
    if skip_mode and ("    def " in line and "on_validation_end" not in line):
        skip_mode = False

    if not skip_mode:
        new_lines.append(line)

with open(file_path, 'w') as f:
    f.writelines(new_lines)

print("✅ lightning.py patched with safe indentation.")

✅ lightning.py patched with safe indentation.


In [ ]:
import os
# Force weights loading fix for PyTorch 2.6
os.environ["TORCH_FORCE_WEIGHTS_ONLY_LOAD"] = "0"

dataset_dir = '/content/drive/MyDrive/colab/piper/'
checkpoint = '/content/drive/MyDrive/colab/piper/Steve/en_US-kushal-medium.ckpt'

!python -m piper_train \
    --dataset-dir "{dataset_dir}" \
    --batch-size 12 \
    --checkpoint-epochs 5 \
    --quality medium \
    --resume_from_checkpoint "{checkpoint}" \
    --accelerator gpu \
    --devices 1 \
    --precision 32 \
    --log_every_n_steps 10 \
    --num_ckpt -1 \
    --max_epochs 3000

DEBUG:piper_train:Namespace(dataset_dir='/content/drive/MyDrive/colab/piper/', checkpoint_epochs=5, quality='medium', resume_from_single_speaker_checkpoint=None, batch_size=12, validation_split=0.1, num_test_examples=5, max_phoneme_ids=None, hidden_channels=192, inter_channels=192, filter_channels=768, n_layers=6, n_heads=2, lr_decay=0.999875, lr_reduce_enabled=False, lr_reduce_factor=0.5, lr_reduce_patience=10, show_plot=False, plot_save_path=None, learning_rate=0.0002, weight_decay=0.01, override_learning_rate=False, grad_clip=None, accelerator='gpu', devices=1, log_every_n_steps=10, max_epochs=3000, seed=1234, random_seed=False, resume_from_checkpoint='/content/drive/MyDrive/colab/piper/Steve/en_US-kushal-medium.ckpt', precision='32', num_ckpt=-1, default_root_dir=None, save_last=None, monitor='val_loss', monitor_mode='min', early_stop_patience=0)
DEBUG:piper_train:Using manual seed: 1234
DEBUG:piper_train:Checkpoints will be saved every 5 epoch(s)
DEBUG:piper_train:-1 Checkpoints w

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
# Note: Don't do this while training is running unless you intend to stop.

Drive not mounted, so nothing to flush and unmount.


In [ ]:
#@markdown # <font color="ffc800"> **6. Train.** 🏋️‍♂️
#@markdown ---
#@markdown ### Run this cell to train your final model!

#@markdown ---
#@markdown ### <font color="orange">**Disable validation?**
#@markdown By uncheck this checkbox, this will allow to train the full dataset, without using any audio files or examples as a validation set. So, it will not be able to generate audios on the tensorboard while it's training. It is recommended to disable validation on extremely small datasets.
validation = False #@param {type:"boolean"}
if validation:
    validation_split = 0.01
    num_test_examples = 1
else:
    validation_split = 0
    num_test_examples = 0
if not save_last:
    save_last_command = ""
else:
    save_last_command = "--save_last True "
get_ipython().system(f'''
python -m piper_train \
--dataset-dir "{output_dir}" \
--accelerator 'gpu' \
--devices 1 \
--batch-size {batch_size} \
--validation-split {validation_split} \
--num-test-examples {num_test_examples} \
--quality {quality} \
--checkpoint-epochs {checkpoint_epochs} \
--num_ckpt {num_ckpt} \
{save_last_command}\
--log_every_n_steps {log_every_n_steps} \
--max_epochs {max_epochs} \
{ft_command}\
--precision 32
''')

DEBUG:piper_train:Namespace(dataset_dir='/content/drive/MyDrive/colab/piper/', checkpoint_epochs=5, quality='medium', resume_from_single_speaker_checkpoint=None, batch_size=12, validation_split=0.0, num_test_examples=0, max_phoneme_ids=None, hidden_channels=192, inter_channels=192, filter_channels=768, n_layers=6, n_heads=2, lr_decay=0.999875, lr_reduce_enabled=False, lr_reduce_factor=0.5, lr_reduce_patience=10, show_plot=False, plot_save_path=None, learning_rate=0.0002, weight_decay=0.01, override_learning_rate=False, grad_clip=None, accelerator='gpu', devices=1, log_every_n_steps=1000, max_epochs=10000, seed=1234, random_seed=False, resume_from_checkpoint='/content/pretrained.ckpt', precision='32', num_ckpt=1, default_root_dir=None, save_last=None, monitor='val_loss', monitor_mode='min', early_stop_patience=0)
DEBUG:piper_train:Using manual seed: 1234
DEBUG:piper_train:Checkpoints will be saved every 5 epoch(s)
DEBUG:piper_train:1 Checkpoints will be saved
GPU available: True (cuda),

#  <font color="orange">**Have you finished training and want to test the model?**

* If you want to run this model in any software that Piper integrates or the same Piper app, export your model using the [model exporter notebook](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_model_exporter.ipynb)!
* Wait! I want to test this right now before exporting it to the supported format for Piper. Test your generated last.ckpt with [this notebook](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ckpt).ipynb)!